## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# Drop Nan Values
df = df.dropna()

In [5]:
## Get the Independent Features
X = df.drop(['id','label'],axis=1)

In [6]:
## Get the Dependent features
y = df['label']

In [7]:
X.shape

(18285, 3)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.8.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
### Vocabulary size
voc_size = 5000

### Onehot Representation

In [13]:
messages = X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
### Dataset Preprocessing

from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
onehot_repr = [one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4716, 2903, 3611, 2591, 3509, 3449, 4463, 3990, 838, 1434],
 [1472, 119, 3826, 4490, 3337, 1708, 3906],
 [4950, 1188, 1690, 887],
 [3252, 4072, 4284, 3861, 4597, 457],
 [2042, 3337, 1444, 4070, 1575, 4200, 3337, 4575, 4857, 4360],
 [4957,
  461,
  3725,
  906,
  4088,
  1385,
  1292,
  3706,
  4411,
  4199,
  2635,
  4380,
  4617,
  3544,
  3906],
 [435, 3669, 879, 17, 3043, 1437, 2766, 4081, 929, 4168, 2450],
 [3955, 3499, 3331, 87, 3238, 1691, 1385, 2058, 929, 4168, 2450],
 [376, 3838, 2279, 3358, 490, 550, 595, 2576, 1385, 3917],
 [894, 4621, 1114, 277, 2357, 3128, 3480, 2781],
 [1068, 551, 378, 4871, 140, 2846, 4798, 160, 669, 621, 1870],
 [3861, 2625, 3509, 550, 1385, 3238],
 [1583, 1645, 4644, 1247, 3315, 1248, 295, 508, 1975],
 [3855, 2903, 2909, 4986, 3161, 2188, 4457, 929, 4168, 2450],
 [713, 3515, 69, 2304, 2177, 929, 4168, 2450],
 [593, 3505, 4344, 27, 1370, 4686, 2966, 3349, 1742, 3811],
 [100, 910, 119],
 [3118, 4369, 3974, 3231, 1385, 1056, 18, 3906],
 [4588, 3787, 3826

### Embedding Representation

In [21]:
sent_length=20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3990  838 1434]
 [   0    0    0 ... 3337 1708 3906]
 [   0    0    0 ... 1188 1690  887]
 ...
 [   0    0    0 ...  929 4168 2450]
 [   0    0    0 ... 1819 1187 2332]
 [   0    0    0 ... 4571 3366 4717]]


In [22]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4716,
       2903, 3611, 2591, 3509, 3449, 4463, 3990,  838, 1434])

In [23]:
## Creating model
embedding_vector_features=40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
len(embedded_docs)

18285

In [25]:
len(y)

18285

In [26]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [27]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [29]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 34ms/step - loss: 0.3370 - accuracy: 0.8348 - val_loss: 0.1967 - val_accuracy: 0.9170
Epoch 2/10
192/192 [==============================] - 5s 25ms/step - loss: 0.1493 - accuracy: 0.9389 - val_loss: 0.1917 - val_accuracy: 0.9220
Epoch 3/10
192/192 [==============================] - 6s 34ms/step - loss: 0.1057 - accuracy: 0.9598 - val_loss: 0.2040 - val_accuracy: 0.9205
Epoch 4/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0796 - accuracy: 0.9708 - val_loss: 0.2285 - val_accuracy: 0.9158
Epoch 5/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0648 - accuracy: 0.9780 - val_loss: 0.2689 - val_accuracy: 0.9152
Epoch 6/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0487 - accuracy: 0.9833 - val_loss: 0.3356 - val_accuracy: 0.9150
Epoch 7/10
192/192 [==============================] - 22s 116ms/step - loss: 0.0378 - accuracy: 0.9882 - val_loss: 0.3706 - val_accuracy: 0.

### Adding Dropout 

In [30]:
from tensorflow.keras.layers import Dropout

In [31]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [32]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [33]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

In [34]:
y_pred

array([[0],
       [0],
       [1],
       ...,
       [1],
       [1],
       [0]])

In [35]:
confusion_matrix(y_test,y_pred)

array([[2638,  781],
       [1297, 1319]], dtype=int64)

In [36]:
accuracy_score(y_test,y_pred)

0.6556752278376139

In [37]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.67      0.77      0.72      3419
           1       0.63      0.50      0.56      2616

    accuracy                           0.66      6035
   macro avg       0.65      0.64      0.64      6035
weighted avg       0.65      0.66      0.65      6035

